In [ ]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import os


import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
import pyro.distributions as dist
import pyro.distributions.constraints as constraints
from sklearn import preprocessing
from sklearn.metrics import r2_score
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)

pyro.__version__

# Rational agent

The idea is to train n models where each model is trained using n trials.

extract valid subjects

In [ ]:
import extract_correct_csv
valid_sub = extract_correct_csv.extract_only_valid_subject()

read real data of subject #2 (data equals to all subjects)

In [ ]:
# read dataset
df = pd.read_csv('data/newLookAtMe/newLookAtMe02.csv')
df_rational = df[['morphing level', 'shock']]
df_rational['shock'] = df_rational['shock'].astype(int) #setting shock as int instead of boolean
df_rational['morphing level'] = [int(d==6) for d in df_rational['morphing level']] # if morphing level==6 -> 1

In [ ]:
data_np = df_rational.to_numpy()

HABITUATION_TRIALS = 16
ACQUISITION_TRIALS = 48
data_all = data_np[16:]
learning_data = data_np[HABITUATION_TRIALS:ACQUISITION_TRIALS] # remove only habituation

In [ ]:
data = torch.tensor(data_all)
N = data.shape[0]

In [ ]:
array_csplus_simulated = np.load('output/pyro/complete_rational/csplus.npy',allow_pickle=True)
array_csminus_simulated = np.load('output/pyro/complete_rational/csminus.npy',allow_pickle=True)
total_array_simulated = np.load('output/pyro/complete_rational/total.npy',allow_pickle=True)

## shock expectancy simulated

In [ ]:
df_ = df[['shock', 'rating', 'morphing level']]
df_['shock'] = df_['shock'].astype(int)
df_['morphing level'] = [int(d == 6) for d in df_['morphing level']]
df_['rating'] = df_['rating'].replace([1, 2, 3, 4, 5], [0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
#remove first 16 trials real data
df_new=df_
#consider only the learning phase
df_learning = df_
df_learning

In [ ]:
y_plus_real = df_new.loc[df_new['morphing level']==1].rating.values
x_plus_real = np.array(df_new.loc[df_new['morphing level']==1].index)
y_minus_real = df_new.loc[df_new['morphing level']==0].rating.values
x_minus_real = np.array(df_new.loc[df_new['morphing level']==0].index)
y_real=df_new.rating.values

y_plus_train = df_learning.loc[df_learning['morphing level']==1].rating.values
x_plus_train = np.array(df_learning.loc[df_learning['morphing level']==1].index)
y_minus_train = df_learning.loc[df_learning['morphing level']==0].rating.values
x_minus_train = np.array(df_learning.loc[df_learning['morphing level']==0].index)
y_train=df_learning.rating.values

In [ ]:
fig = plt.figure(figsize=(13,6))
plt.title('Analysis of the shock prediction comparing subject #2 and rational agent')
plt.scatter(x_plus_real, y_plus_real, color='blue', label='cs+ real')
plt.scatter(array_csplus_simulated[:,0], array_csplus_simulated[:,1], color='darkblue', label='cs+ simulated')
plt.scatter(x_minus_real, y_minus_real, color='red', label='cs- real')
plt.scatter(array_csminus_simulated[:,0], array_csminus_simulated[:,1], color='darkred', label='cs- simulated')
plt.legend(loc='right')
plt.axvline(x=16, linestyle='--', color='green')
plt.axvline(x=48, linestyle='--', color='green')
plt.xlabel('Trial')
plt.ylabel('P(condition | visual stimulus)')
plt.show()

# correlation

## analysis of the correlation between rational agent and real subject with pearson

In [ ]:
# rating between 16-160 trial rational agent
rating_rational = total_array_simulated

In [ ]:
df_correlation = pd.DataFrame(columns=['subject','pearson','r2score'])

for sub in valid_sub:
    subj_ = extract_correct_csv.read_correct_subject_csv(sub)
    df_sub = pd.read_csv('data/newLookAtMe/newLookAtMe'+subj_+'.csv')
    df_sub = df_sub[['shock', 'rating', 'morphing level']]
    df_sub['shock'] = df_sub['shock'].astype(int)
    df_sub['morphing level'] = [int(d == 6) for d in df_sub['morphing level']]
    df_sub['rating'] = df_sub['rating'].replace([1, 2, 3, 4, 5], [0.2, 0.4, 0.6, 0.8, 1])
    df_sub_learn = df_sub[16:]
    rating_sub = np.array(df_sub_learn['rating'])

    # remove nan
    bad = ~np.logical_or(np.isnan(rating_sub), np.isnan(rating_rational))
    rating_sub_ = np.compress(bad, rating_sub)
    rating_rational_ = np.compress(bad, rating_rational)

    #pearson corr coeff
    pearson = round(np.corrcoef(rating_sub_,rating_rational_)[0][1],2)

    #r2 score
    r2 = round(r2_score(rating_sub_,rating_rational_),2)
    df_tmp = pd.DataFrame({'subject':sub,'pearson':pearson,'r2score':r2},index=np.arange(1))
    df_correlation = pd.concat([df_correlation,df_tmp])


sias_df = pd.read_csv('data/sias_score.csv').drop(columns='social_anxiety')
sias_df['subject'] = [float(x) for x in sias_df['subject']]

lds_df = pd.read_csv('data/lds_subjects.csv')
lds_df['subject'] = [float(x) for x in lds_df['subject']]

df_correlation['subject'] = [float(x) for x in df_correlation['subject']]

df_corr_ = pd.concat([sias_df.set_index('subject'), lds_df.set_index('subject'), df_correlation.set_index('subject')], axis=1).reset_index()

# drop nan
lds_df['subject'] = [float(x) for x in lds_df['subject']]

In [ ]:
merged_df = pd.concat([sias_df.set_index('subject'), lds_df.set_index('subject'), df_correlation.set_index('subject')], axis=1)
merged_df.reset_index()

In [ ]:
df_corr = df_correlation.join(lds_df, lsuffix='', rsuffix='_other',how='inner').drop(columns='subject_other')
df_corr_ = df_corr.join(sias_df,lsuffix='', rsuffix='_other',how='inner').drop(columns='subject_other')
df_corr_

In [ ]:
df_corr_notna = df_corr_[df_corr_['pearson'].notna()]
df_corr_notna = df_corr_notna[df_corr_['lds'].notna()]
df_corr_notna = df_corr_notna[df_corr_['sias_score'].notna()]

#extract subject with high value lds and lower lds
lds_ = df_corr_notna.sort_values('lds').reset_index().drop(columns='index')
lower_lds = lds_[:7]
higher_lds = lds_[-7:]

# Analysis subject with more fear gen vs subjects less fear gen

In [ ]:
sias_ = df_corr_notna.sort_values('sias_score',ascending=True).reset_index().drop(columns='index')
lower_sias = sias_[:5]
sias_ = df_corr_notna.sort_values('sias_score',ascending=False).reset_index().drop(columns='index')
higher_sias= sias_[:5]

# Analysis subject more fear gen vs less fear gen

In [65]:
l_low_p = lower_lds['pearson'].median()
l_hig_p = higher_lds['pearson'].median()
l_low_r = lower_lds['r2score'].median()
l_hig_r = higher_lds['r2score'].median()

print('Correlation between 7 more/less fear gen subjects with the rational agent\n')
print('Pearson\nHigh fear gen: ',l_low_p, ' Low fear gen:',l_hig_p)
print('\nR2score\nHigh fear gen: ',l_low_r, ' Low fear gen:',l_hig_r)

0.46

# Analysis subject more anxiety vs less anxiety

In [ ]:
s_low_p = lower_sias['pearson'].median()
s_hig_p = higher_sias['pearson'].median()

s_low_r = lower_sias['r2score'].median()
s_hig_r = higher_sias['r2score'].median()

print('Correlation between 5 more/less anxiety subjects with the rational agent\n')
print('Pearson\nHigh anxiety: ',s_hig_p, ' Low anxiety:',s_low_p)
print('\nR2score\nHigh anxiety: ',s_hig_r, ' Low anxiety:',s_low_r)

# Analysis subject with more anxiety vs subjects less anxiety

In [ ]:
sias_

In [80]:
lower_sias['pearson'].median()

0.6

In [ ]:
higher_sias['pearson'].median()

# Analysis sliding window K

In [ ]:
k_wind = [2, 5, 10, 25, 50, 100, 150]
for k in list(k_wind):

    array_csplus_simulated = np.load('output/pyro/sliding_wind/k'+str(k)+'_csplus.npy',allow_pickle=True)
    array_csminus_simulated = np.load('output/pyro/sliding_wind/k'+str(k)+'_csminus.npy',allow_pickle=True)
    total_array_simulated = np.load('output/pyro/sliding_wind/k'+str(k)+'_total.npy',allow_pickle=True)

    rating_rational = total_array_simulated
    rating_rational = rating_rational[16:]

    df_correlation = pd.DataFrame(columns=['subject','pearson'])

    for sub in valid_sub:
        subj_ = extract_correct_csv.read_correct_subject_csv(sub)
        df_sub = pd.read_csv('data/newLookAtMe/newLookAtMe'+subj_+'.csv')
        df_sub = df_sub[['shock', 'rating', 'morphing level']]
        df_sub['shock'] = df_sub['shock'].astype(int)
        df_sub['morphing level'] = [int(d == 6) for d in df_sub['morphing level']]
        df_sub['rating'] = df_sub['rating'].replace([1, 2, 3, 4, 5], [0.2, 0.4, 0.6, 0.8, 1])
        df_sub_learn = df_sub[16:]
        rating_sub = np.array(df_sub_learn['rating'])
        bad = ~np.logical_or(np.isnan(rating_sub), np.isnan(rating_rational))
        rating_sub_ = np.compress(bad, rating_sub)
        rating_rational_ = np.compress(bad, rating_rational)
        pearson = round(np.corrcoef(rating_sub_,rating_rational_)[0][1],2)
        df_correlation = df_correlation.append({'subject':sub,'pearson':pearson},ignore_index=True)
    sias_df = pd.read_csv('data/sias_score.csv').drop(columns='social_anxiety')
    sias_df['subject'] = [float(x) for x in sias_df['subject']]

    lds_df = pd.read_csv('data/lds_subjects.csv')

    lds_df['subject'] = [float(x) for x in lds_df['subject']]
    df_corr = df_correlation.join(lds_df, lsuffix='', rsuffix='_other',how='inner').drop(columns='subject_other')
    df_corr_ = df_corr.join(sias_df,lsuffix='', rsuffix='_other',how='inner').drop(columns='subject_other')
    df_corr_notna = df_corr_[df_corr_['pearson'].notna()]
    df_corr_notna = df_corr_notna[df_corr_['lds'].notna()]
    df_corr_notna = df_corr_notna[df_corr_['sias_score'].notna()]
    lds_ = df_corr_notna.sort_values('lds').reset_index().drop(columns='index')
    lower_lds = lds_[:7]
    higher_lds = lds_[-7:]

    print('\n\n--------------- K = '+str(k)+' ---------------')

    print('7 subj less lds corr with rational agent:',lower_lds['pearson'].median())
    print('7 subj more lds corr with rational agent:',higher_lds['pearson'].median())

    sias_ = df_corr_notna.sort_values('sias_score',ascending=True).reset_index().drop(columns='index')
    lower_sias = sias_[:7]
    sias_ = df_corr_notna.sort_values('sias_score',ascending=False).reset_index().drop(columns='index')
    higher_sias= sias_[:7]
    print()
    print('7 subj less sias corr with rational agent:',lower_sias['pearson'].median())
    print('7 subj more sias corr with rational agent:',higher_sias['pearson'].median())

In [ ]:
df_corr_notna['sias_score'] = [round(x,2) for x in preprocessing.normalize([df_corr_notna['sias_score']])[0]]
df_corr_notna['lds'] = [round(x,2) for x in preprocessing.normalize([df_corr_notna['lds']])[0]]
df_corr_notna['pearson'] = [round(x,2) for x in preprocessing.normalize([df_corr_notna['pearson']])[0]]


In [ ]:
df_corr_notna['subject'] = [int(x) for x in df_corr_notna['subject']]

In [ ]:
lds_ = df_corr_notna.sort_values('lds').reset_index().drop(columns='index')
lower_lds = lds_[:5]
higher_lds = lds_[-5:]

In [ ]:
subjects = tuple(df_corr_notna['subject'])

In [ ]:
values = {'pearson':(),'lds':(),'sias':()}
for index,row in df_corr_notna.iterrows():
    new_p = values['pearson'] + (row['pearson'],)
    values.update({'pearson':new_p})
    new_l = values['lds'] + (row['lds'],)
    values.update({'lds':new_l})
    new_s = values['sias'] + (row['sias_score'],)
    values.update({'sias':new_s})

In [ ]:
values_no_sias = {'pearson':(),'lds':()}
for index,row in df_corr_notna.iterrows():
    new_p = values_no_sias['pearson'] + (row['pearson'],)
    values_no_sias.update({'pearson':new_p})
    new_l = values_no_sias['lds'] + (row['lds'],)
    values_no_sias.update({'lds':new_l})

In [ ]:
subjects = tuple(df_corr_notna['subject'])
values_no_lds = {'pearson': (), 'sias': ()}
for index, row in df_corr_notna.iterrows():
    new_p = values_no_lds['pearson'] + (row['pearson'],)
    values_no_lds.update({'pearson': new_p})
    new_s = values_no_lds['sias'] + (row['sias_score'],)
    values_no_lds.update({'sias': new_s})

In [ ]:
x = np.arange(len(subjects))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15,8),constrained_layout=True)

for attribute, measurement in values_no_lds.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    #ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('correlation attributes by subjects')
ax.set_xticks(x + width, subjects)
ax.legend(loc='upper left', ncols=3)

plt.show()

In [ ]:
x = np.arange(len(subjects))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15,8),constrained_layout=True)

for attribute, measurement in values_no_sias.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    #ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('correlation attributes by subjects')
ax.set_xticks(x + width, subjects)
ax.legend(loc='upper left', ncols=3)

plt.show()

In [ ]:
x = np.arange(len(subjects))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15,8),constrained_layout=True)

for attribute, measurement in values.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    #ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('correlation attributes by subjects')
ax.set_xticks(x + width, subjects)
ax.legend(loc='upper left', ncols=3)

plt.show()

# rational agent sliding window

The idea is to have a rational agent with a limited memory over previous trials.
k = param sliding window dimension


In [ ]:
import extract_correct_csv
valid_sub = extract_correct_csv.extract_only_valid_subject()

In [ ]:
# read dataset
df = pd.read_csv('data/newLookAtMe/newLookAtMe02.csv')
df_rational = df[['morphing level', 'shock']]
df_rational['shock'] = df_rational['shock'].astype(int) #setting shock as int instead of boolean
df_rational['morphing level'] = [int(d==6) for d in df_rational['morphing level']] # if morphing level==6 -> 1

In [ ]:
data_np = df_rational.to_numpy()


def counter_window(data, k=0):
    N = data.shape[0]
    counter = torch.zeros((N,4))
    for i in range(len(data)):
        dict_ = {'[0 0]':0, '[0 1]': 0, '[1 0]':0, '[1 1]':0}
        if k == 0 or k > i:
            tmp_data = data[:i+1]
        else:
            tmp_data = data[i-k:i+1]
            #print('im here')
        # count occurencies
        for x in tmp_data:
            dict_[str(x)] += 1
        values = np.array(list(dict_.values()))
        counter[i] = torch.tensor(values)
    return counter

In [ ]:
counter = counter_window(data_np, 3)

counter = counter.reshape((len(data_np), 2, 2))
counter.shape

In [ ]:
# categorical/multinomial distribution

# uniform prior
prior_counts = torch.ones((2,2))


#model
def model(data):
    prior = pyro.sample("prior", dist.Dirichlet(prior_counts))
    total_counts = int(data.sum())
    pyro.sample("likelihood", dist.Multinomial(total_counts, prior), obs=data)


nuts_kernel = NUTS(model)
num_samples, warmup_steps = (300, 200)

mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps, disable_progbar=True)
all_means = []

# sampling
for i in range(len(counter)):
    mcmc.run(counter[i])
    hmc_samples = {k: v.detach().cpu().numpy()
                   for k, v in mcmc.get_samples().items()}
    means = hmc_samples['prior'].mean(axis=0)
    stds = hmc_samples['prior'].std(axis=0)
    print('observation: ', data_np[i])
    print('probabilities: ', means)
    all_means.append(means)

## Deep Markov model

![example](https://pyro.ai/examples/_static/img/model.png)

In [ ]:
import argparse
import logging
import time
from os.path import exists
import numpy as np
import torch
import torch.nn as nn
import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.distributions import TransformedDistribution
from pyro.distributions.transforms import affine_autoregressive
from pyro.infer import (
    SVI,
    JitTrace_ELBO,
    Trace_ELBO,
    TraceEnum_ELBO,
    TraceTMC_ELBO,
    config_enumerate,
)
from pyro.optim import ClippedAdam

In [ ]:
# emission function (Emit in the figure)
class Emitter(nn.Module):
    """
    Parameterizes the bernoulli observation likelihood p(x_t|z_t)

    """
    def __init__(self, input_dim, z_dim, emission_dim):
        super().__init__()
        # emission_dim is the number of hidden units in the neural network
        # three linear transformations used in the neural network
        self.lin_z_to_hidden = nn.Linear(z_dim, emission_dim)
        self.lin_hidden_to_hidden = nn.Linear(emission_dim, emission_dim)
        self.lin_hidden_to_input = nn.Linear(emission_dim, input_dim)
        # two non linear used in the neural network
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z_t):
        """
        Given the latent z at a particular time step t, we return the vector of probabilities 'ps' that parametrizes the bernoulli distribution p(x_t|z_t)
        Taken together the elements of ps encode which notes we expect to observe at time t given the state of the system (as encoded in z_t).
        """
        h1 = self.relu(self.lin_z_to_hidden(z_t))
        h2 = self.relu(self.lin_hidden_to_hidden(h1))
        ps = self.sigmoid(self.lin_hidden_to_input(h2))
        return ps

In [ ]:
# gated transition (Trans in the figure above)
class GatedTransition(nn.Module):
    """
    Parameterizes the gaussian latent transition probability p(z_t | z_{t-1})
    """
    def __init__(self, z_dim, transition_dim):
        super().__init__()
        # six linear transform
        self.lin_gate_z_to_hidden = nn.Linear(z_dim, transition_dim)
        self.lin_gate_hidden_to_z = nn.Linear(transition_dim, z_dim)
        self.lin_proposed_mean_z_to_hidden = nn.Linear(z_dim, transition_dim)
        self.lin_proposed_mean_hidden_to_z = nn.Linear(transition_dim, z_dim)
        self.lin_sig = nn.Linear(z_dim, z_dim)
        self.lin_z_to_loc = nn.Linear(z_dim, z_dim)

        # modify the default initialization of lin_z_to_loc
        # so that it's starts out as the IDENTITY FUNCTION
        self.lin_z_to_loc.weight.data = torch.eye(z_dim)
        self.lin_z_to_loc.bias.data = torch.zeros(z_dim)

        # three non linear
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()


    def forward(self, z_t_1):
        """
        Given the latent z_{t-1} corresponding to the time step t-1 we return the mean and scale vectors that parameterize the
        (diagonal) gaussian distribution p(z_t | z_{t-1})
        """
        # compute the gating function
        _gate = self.relu(self.lin_gate_z_to_hidden(z_t_1))
        gate = self.sigmoid(self.lin_gate_hidden_to_z(_gate))

        # compute the 'proposed mean'
        _proposed_mean = self.relu(self.lin_proposed_mean_z_to_hidden(z_t_1))
        proposed_mean = self.lin_proposed_mean_hidden_to_z(_proposed_mean)

        # assemble the actual mean used to sample z_t, which mixes a linear transformation of z_{t-1} with the proposed mean
        # modulated by the gating function
        loc = (1 - gate) * self.lin_z_to_loc(z_t_1) + gate * proposed_mean

        # compute the scale used to sample z_t, using the proposed
        # mean from above as input. the softplus ensures that scale is positive
        scale = self.softplus(self.lin_sig(self.relu(proposed_mean)))

        # return loc, scale which can be fed into Normal (mean and covariance of our Gaussian)
        return loc, scale

In [ ]:
# simple model (not working)
def model(...):
    # initializer
    z_prev = self.z_0

    # sample the latents z and observed x's one time step at a time
    for t in range(1, T_max + 1):
        # the next two lines of code sample z_t ~ p(z_t | z_{t-1}).
        # first compute the parameters of the diagonal gaussian
        # distribution p(z_t | z_{t-1})
        z_loc, z_scale = self.trans(z_prev)
        # then sample z_t according to dist.Normal(z_loc, z_scale)
        z_t = pyro.sample("z_%d" % t, dist.Normal(z_loc, z_scale))

        # compute the probabilities that parameterize the bernoulli likelihood
        emission_probs_t = self.emitter(z_t)
        # the next statement instructs pyro to observe x_t according to the
        # bernoulli distribution p(x_t|z_t)
        pyro.sample("obs_x_%d" % t,
                    dist.Bernoulli(emission_probs_t),
                    obs=mini_batch[:, t - 1, :])
        # the latent sampled at this time step will be conditioned upon
        # in the next time step so keep track of it
        z_prev = z_t

### Agent example

To define a hidden Markov Model (HMM) for this scenario, we need to identify the key components of an HMM:
1.	States: In our case, the state is whether the agent is in a shocked state or not. We will use 0 to represent a non-shocked state and 1 to represent a shocked state.
2.	Observations: The observation is whether the agent sees an aggressive or non-aggressive image. We will use 0 to represent a non-aggressive image and 1 to represent an aggressive image.
3.	Transition probabilities: The transition probabilities determine the probability of moving from one state to another. In our case, the transition probabilities will depend on the current state and the shock received. Specifically, if the agent is in a non-shocked state and receives a shock, there is a high probability of transitioning to the shocked state. Conversely, if the agent is in the shocked state and does not receive a shock, there is a high probability of transitioning to the non-shocked state.
4.	Emission probabilities: The emission probabilities determine the probability of observing a particular image given the current state. In our case, the emission probabilities will depend on the current state. Specifically, if the agent is in the non-shocked state, there is a high probability of observing a non-aggressive image, and if the agent is in the shocked state, there is a high probability of observing an aggressive image.

In [ ]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import Trace_ELBO, SVI


def transition_model(state):
    p_stay = 0.7 if state else 0.9
    p_switch = 1 - p_stay
    return pyro.sample('state_transition', dist.Categorical(torch.tensor([p_stay, p_switch])))


def emission_model(state):
    p_emission = 0.9 if state else 0.1
    return pyro.sample('emission', dist.Bernoulli(torch.tensor(p_emission)))


def initial_model():
    return pyro.sample('initial_state', dist.Categorical(torch.tensor([0.5, 0.5])))


def model(stimuli, shock, initial_state):
    states = [pyro.sample('init_state', dist.Categorical(initial_state))]
    emissions = [emission_model(states[0]).item()]
    for i in range(1, len(stimuli)):
        states.append(transition_model(states[i-1]).item())
        emissions.append(emission_model(states[i]).item())
    return states, emissions


def guide(stimuli, shock, initial_state, states, emissions):
    for i in range(len(stimuli)):
        states[i] = pyro.sample('state_{}'.format(i), dist.Bernoulli(torch.tensor(0.5)))
        emissions[i] = pyro.sample('emission_{}'.format(i), dist.Bernoulli(torch.tensor(0.5)))


stimuli = torch.tensor([0, 0, 0, 1, 1, 0, 1, 1, 0])
shock = torch.tensor([0, 0, 0, 1, 1, 0, 0, 1, 0])
initial_state = torch.tensor([0.5, 0.5])

optimizer = pyro.optim.Adam({'lr': 0.1})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

for i in range(1000):
    states, emissions = model(stimuli, shock, initial_state)
    loss = svi.step(stimuli, shock, initial_state, states, emissions)
    if i % 100 == 0:
        print("step {}: loss = {}".format(i, loss))


## Rational agent discretisation

In [ ]:
valid_subjects = extract_correct_csv.extract_only_valid_subject()

In [ ]:
total_array_simulated = np.load('output/pyro/complete_rational/total.npy',allow_pickle=True)
total_array_simulated

In [ ]:
values = np.array([0, 0.2, 0.4, 0.6, 0.8, 1])
discretized_data = np.digitize(total_array_simulated, values)
print(discretized_data.shape)


df_global = pd.DataFrame(columns=['Subject', 'Rating rational', 'Rating real'])

for sub in valid_sub:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_sub = pd.read_csv('data/newLookAtMe/newLookAtMe'+string_sub+'.csv')
    df_sub = df_sub[16:]
    tmp_df = pd.DataFrame({'Subject': sub, 'Rating rational': discretized_data, 'Rating real': df_sub['rating']})
    df_global = pd.concat([df_global, tmp_df])

df_global = df_global.reset_index(drop=True)

In [ ]:
# useless up to now
'''df_global = df_global.groupby('Subject', as_index=False).agg({'Rating rational': lambda x: x.tolist(), 'Rating real': lambda x: x.tolist()})
df_global['Rating rational'] = df_global['Rating rational'].apply(lambda x: np.array(x))
df_global['Rating real'] = df_global['Rating real'].apply(lambda x: np.array(x))'''

In [ ]:

df_global

In [ ]:
from pyirr import read_data, intraclass_correlation
data = read_data("anxiety")  # loads example data
intraclass_correlation(data, "twoway", "agreement")

In [ ]:
df_sub_ = df_global[df_global.Subject == 5].dropna().drop(columns=['Subject']).reset_index(drop=True)
df_sub_['Rating rational'] = df_sub_['Rating rational'].astype(float)
intraclass_correlation(df_sub_)